# Degree-hour estimation of energy savings by intermittent heating

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/cghiaus/dm4bem_book/HEAD?labpath=%2Ftutorials%2FA04Degree_hours_intermittance.ipynb)

Intermittence consists in reducing the heating power during the night. In a first approximation, the savings obtained can be estimated by using degree-hours ([Ghiaus and Allard, 2006](https://doi.org/10.1016/j.solener.2005.05.019)).

## Degree-hour method

Let's model the building as a volume at indoor temperature separated from the environment by an envelope. Indoor and outdoor temperatures and the properties of the envelope are homogeneous. The heat balance in steady-state is:

$$\dot{Q}_{HVAC} - U_{bldg}·S·(\theta_{in} - \theta_{out}) = 0$$

where:
- $\dot{Q}_{HVAC}$ - power delivered by the Heating, Ventilation and Air Conditioning (HVAC) system to the building to compensate the heat losses in order to maintain the indoor temperature at value $\theta_{in}$, W;
- $U_{bldg}·S·(\theta_{in} - \theta_{out})$ - heat losses of the building, W,

    where:

    - $U_{bldg}$ - overall heat loss coefficient of the building, W/(m²·K) ([Ghiaus and Alzetto, 2019](https://doi.org/10.1080/19401493.2018.1561753));
    - $S$ - surface area of the building envelope, m²;
    - $\theta_{out}$ - outdoor temperature, °C;
    - $\theta_{in}$ - base (indoor) temperature, °C.

The energy consumption of the HVAC system needed to maintain the indoor temperature at the value $\theta_{in}$ over a period of time is the integral in time of the power load ([Ghiaus, 2006](https://doi.org/10.1016/j.enbuild.2005.08.014)):

$$Q_{HVAC} = \int_{0}^{t} |\dot{Q}_{HVAC}| dt = \int_{0}^{t} |U_{bldg}·S·(\theta_{in} - \theta_{out})| dt $$

where $|\dot{Q}_{HVAC}|$ is the absolute value of $\dot{Q}_{HVAC}$.

By considering that the overall heat loss coefficient $U_{bldg}$ is constant, then ([Ghiaus, 2003](https://doi.org/10.1016/S0378-7788(02)00110-X))

$$Q_{HVAC} = U_{bldg}·S· \int_{0}^{t} |\theta_{in} - \theta_{out}| dt $$

which implies that the energy consumption is proportional to $\int_{0}^{t} |\theta_{in} - \theta_{out}| dt$. This integral can be discretized (in time):

$$\int_{0}^{t} [\theta_{in} - \theta_{out}| dt  \approx  \sum_{k=0}^{n} |\theta_{in,k} - \theta_{out,k}| \Delta t$$

where:
- $\theta_{out,k}$ - mean outdoor temperature in the time interval $\Delta t$ at discrete time (step) $k$, °C;
- $\theta_{in,k}$ - mean indoor temperature in the time interval $\Delta t$ at discrete time (step) $k$, °C;

The expression $\sum_{k=0}^{n} |\theta_{out,k} - \theta_{in,k}| \Delta t$ is called:
- _degree-days_ when the discrete time $\Delta t$ is one day;
- _degree-hours_ when the discrete time $\Delta t$ is one hour.

Degree-days and degree-hours for heating and cooling are given in `.stat` file of _expanded EnergyPlus weather statistics_ that can be douwloaded from [Climate.OneBuilding.Org](https://climate.onebuilding.org/default.html).

Degree-days and degree-hours are used to estimate the influence of climate and of indoor temperature on energy consumption ([Ghiaus, 2003](https://doi.org/10.1016/S0378-7788(02)00110-X), [Ghiaus and Allard, 2006](https://doi.org/10.1016/j.solener.2005.05.019), [Ghiaus et al., 2006](https://doi.org/10.1016/j.buildenv.2005.02.003)). The method has the advantage of comparing the influence of different climates and indoor temperatures without knowing the actual building. It has the disadvantage of not taking into account properly the gains (solar and internal), the variable ventilation rates, and the distribution of outdoor temperatures (e.g., air and ground). Therefore, it is suited for comparing the potential of energy savings due to climate and indoor temperature.

## Estimation of energy used for heating with degree-hours method

The Heating, Ventilation and Air Conditioning (HVAC) system controls the indoor temperature all the time by heating and cooling the building. Heating is needed only when the outdoor temperature is lower than the indoor temperature. The energy consumption for heating is proportional to degree-hours (or degree-days) for heating:

$$Q_h \sim  \sum_{k=0}^{n} (\theta_{in,k} - \left \lceil  \theta_{out,k} \right \rceil) \Delta t $$

where:
- $Q_h$ - energy needed for heating, J;
- $\sum_{k=0}^{n} (\theta_{int,k} - \left \lceil  \theta_{out,k} \right \rceil) \Delta t$ - degree-hours for heating;
- $\theta_{in,k}$ - indoor temperature over the time $k$, °C;
- $\left \lceil  \theta_{out,k} \right \rceil$ - outdoor temperature upper bounded by the indoor temperature (i.e., not larger than the indoor temperature), °C,

where
$\left \lceil  \theta_{out,k} \right \rceil = 
\begin{cases}
\theta_{out,k} \text{, if } 
\theta_{out,k} < \theta_{in,k}  
\\
\theta_{in,k} \text{, otherwise}
\end{cases}$

## Degree-hours for intermittent heating
In intermittent heating, the indoor temperature $\theta_{in,k}$ varies in time. The estimation of energy savings by using degree-hours needs to be corrected to take into account the inertia of the building, the gains (solar and internal) and the air infiltration rate. Nevertheless, the degree-hour method gives an indication on savings by indoor temperature setpoint and climate, without modelling the building.

## Python implementation

### Weather data

Download the weather file with extension `.epw` from: 
- [Climate.OneBuilding.Org](http://climate.onebuilding.org): folder of files at the repository of free climate data for building performance simulation,
- [EnergyPlus™](https://energyplus.net/weather): interactive map with locations,
- [LadyBug Tools](http://www.ladybug.tools/epwmap/): interactive map with locations,
- [PV GIS](https://re.jrc.ec.europa.eu/pvg_tools/en/tools.html): interactive map with interpolated data.

For example, for the airport Lyon-Bron, France (N45.73, E5.08), download the files:
- `FRA_Lyon.074810_IWEC.epw` or 
- `FRA_AR_Lyon-Bron.AP.074800_TMYx.2004-2018`

from [Climate.OneBuilding.Org](http://climate.onebuilding.org) and place them in the `./weather_data` folder.

### Inputs
- `filename` file name of weather data;
- `θ` indoor temperature considered constant all the period;
- `θday` indoor temperature during the daytime;
- `θnight` indoor temperature during the nighttime;
- `period_start` day of the year for the start of the period of the degree-hour estimation (note that the year `2000`is a convention);
- `period_end` day of the year for the end of the period of the degree-hour estimation (note that the year `2000`is a convention);
- `daytime_start` time at which the day starts;
- `daytime_end` time at which the day ends.

Note that the daytime needs to be between 00:00 and 24:00. The daytime period cannot include 24:00 (midnight), e.g., it cannot be from 22:00 to 06:00.

In [1]:
import pandas as pd
from dm4bem import read_epw

# Inputs
# ======
filename = '../weather_data/FRA_Lyon.074810_IWEC.epw'

θ = 20          # °C, indoor temperature all time
θday = θ        # °C, indoor temperature during day,, e.g.: 06:00 - 22:00
θnight = 16     # °C, indoor temperature during night 23:00 - 06:00

period_start = '2000-01-01'
period_end = '2000-12-31'

daytime_start = '06:00:00+01:00'
daytime_end = '22:00:00+01:00'

In [3]:
# Computation
# ===========
# read Energy Plus Weather data (file .EPW)
[data, meta] = read_epw(filename, coerce_year=2000)

# select outdoor air temperature; call it θout
df = data[["temp_air"]]
del data
df = df.rename(columns={'temp_air': 'θout'})

# Select the data for a period of the year
df = df.loc[period_start:period_end]

### Temperature differences for constant indoor temperature

$$\Delta \theta_{fix,k} = \theta - \left \lceil  \theta_{out,k} \right \rceil$$

where:
- $\theta$ - indoor temperature (constant all the time), °C;
- $\left \lceil  \theta_{out,k} \right \rceil$ - outdoor temperature bounded by the indoor temperature (i.e., not larger than the indoor temperature), °C, 

where
$\left \lceil  \theta_{out,k} \right \rceil = 
\begin{cases}
\theta_{out,k} \text{, if } \theta_{out,k} < \theta
\\
\theta \text{ otherwise}
\end{cases}$


In [4]:
# Compute degree-hours for fixed set-point
# ----------------------------------------
df['Δθfix'] = θ - df['θout'].where(
    df['θout'] < θ,
    θ)

Δθfix  θout
2000-01-01 00:00:00+01:00   19.2   0.8
2000-01-01 01:00:00+01:00   20.6  -0.6
2000-01-01 02:00:00+01:00   21.5  -1.5
2000-01-01 03:00:00+01:00   21.9  -1.9
2000-01-01 04:00:00+01:00   22.1  -2.1
...                          ...   ...
2000-05-01 19:00:00+01:00    3.0  17.0
2000-05-01 20:00:00+01:00    5.0  15.0
2000-05-01 21:00:00+01:00    6.2  13.8
2000-05-01 22:00:00+01:00    6.6  13.4
2000-05-01 23:00:00+01:00    7.0  13.0

[2904 rows x 2 columns]

### Temperature differences for variable indoor temperature

#### Daytime and nighttime

Daytime is from `day_start` to `day_end`(indicated in `HH:00`). Nighttime is when there is no daytime.

In [6]:
# Define start time for day and night
day_start = pd.to_datetime(daytime_start).time()
day_end = pd.to_datetime(daytime_end).time()

# Daytime should be between 00:00 and 24:00
# Daytime including midnight is not allowed, e.g., 22:00 till 06:00
day = (df.index.time >= day_start) & (df.index.time <= day_end)
night = ~day

#### Temperature differences for daytime

$$\Delta \theta_{day,k} = \theta_{day} - \left \lceil  \theta_{out,k} \right \rceil_{day}$$

where:
- $\theta_{day}$ - indoor temperature over the time $k$, °C;
- $\left \lceil  \theta_{out,k} \right \rceil_{day}$ - outdoor temperature bounded by indoor temperature during the daytime, °C,

where 
$\left \lceil  \theta_{out,k} \right \rceil_{day} =
\begin{cases}
\theta_{out,k} \text{, if } \theta_{out,k} < \theta_{day} \text{ and daytime}
\\
\theta_{day} \text{, otherwise}
\end{cases}$ 


In [7]:
# Degree-hours for daytime
df['Δθday'] = θday - df['θout'].where(
    (df['θout'] < θday) & day,
    θday)

#### Temperature differences for nighttime

$$\Delta \theta_{night,k} = \theta_{night} - \left \lceil  \theta_{out,k} \right \rceil_{night}$$

where:
- $\theta_{night}$ - indoor temperature over the time $k$, °C;
- $\left \lceil  \theta_{out,k} \right \rceil_{night}$ - outdoor temperature bounded by the indoor temperature during the nighttime, °C, 

where
$\left \lceil  \theta_{out,k} \right \rceil_{night} =
\begin{cases}
\theta_{out,k} \text{, if }  \theta_{out,k} < \theta_{night}  \text{ and nighttime}
\\
\theta_{night} \text{, otherwise}
\end{cases}$ 

In [8]:
# Degree-hours for nighttime
df['Δθnight'] = θnight - df['θout'].where(
    (df['θout'] < θnight) & night,
    θnight)

### Degree-hours for heating (DHH)

The degree hours for heating (_DHH_) are the sum of temperature differences:
- for fixed indoor temperature

$$DHH_{fix} = \sum_{k=0}^{n} \Delta \theta_{fix,k} = \sum_{k=0}^{n} ( \theta - \left \lceil  \theta_{out,k} \right \rceil)$$

- for daytime indoor temperature

$$DHH_{day} = \sum_{k=0}^{n} \Delta \theta_{day,k} = \sum_{k=0}^{n} ( \theta_{day} - \left \lceil  \theta_{out,k} \right \rceil_{day})$$

- for nighttime indoor temperature

$$DHH_{night} = \sum_{k=0}^{n} \Delta \theta_{night,k} = \sum_{k=0}^{n} ( \theta_{night} - \left \lceil  \theta_{out,k} \right \rceil_{night})$$

- for intermittence

$$DHH_{interm} = DHH_{day} + DHH_{night}$$

The percentage of energy savings by using intermittent heating is:

$$s = \frac{DHH_{fix} - DHH_{interm}}{DHH_{fix}} 100 \%$$

In [9]:
# Sum of degree-hours for fixed indoor temperature
DHH_fix = df['Δθfix'].sum()

# Sum of degree-hours for intermittent heating
DHH_interm = df['Δθday'].sum() + df['Δθnight'].sum()

# Results
# =======
print(f"degree-hours fixed set-point: {DHH_fix:.1f} h·K")
print(f"degree-hours variable set-point: {DHH_interm:.1f} h·K")
print(f"Estimated savings: {(DHH_fix - DHH_interm) / DHH_fix * 100:.0f} %")

degree-hours fixed set-point: 31.7 h·K
degree-hours variable set-point: 35966.0 h·K
Estimated savings: -113357 %


Note that the obtained percentage of savings is an upper bound. The heating load during the daytime would probably be reduced by free gains, such as solar radiation, occupants and electrical devices. The set-point considered for indoor temperature during nighttime will be attended later than indicated in the time schedule for day/night setpoint due to thermal inertia. The optimal load of intermittently heated building can be assessed by estimating the command of a Model Predictive Controller ([Ghiaus and Hazyuk, 2010](https://doi.org/10.1016/j.enbuild.2010.02.017)).

## References
- [Ghiaus, C. (2003)](https://doi.org/10.1016/S0378-7788(02)00110-X). Free-running building temperature and HVAC climatic suitability. Energy and Buildings, 35(4), 405-411.

- [Ghiaus, C., & Allard, F. (2006)](https://doi.org/10.1016/j.solener.2005.05.019). Potential for free-cooling by ventilation. Solar Energy, 80(4), 402-413.

- [Ghiaus, C., Allard, F., Santamouris, M., Georgakis, C., & Nicol, F. (2006)](https://doi.org/10.1016/j.buildenv.2005.02.003). Urban environment influence on natural ventilation potential. Building and environment, 41(4), 395-406.

- [Ghiaus, C. (2006)](https://doi.org/10.1016/j.enbuild.2005.08.014). Experimental estimation of building energy performance by robust regression. Energy and buildings, 38(6), 582-587.

- [Ghiaus, C., & Hazyuk, I. (2010)](https://doi.org/10.1016/j.enbuild.2010.02.017). Calculation of optimal thermal load of intermittently heated buildings. Energy and Buildings, 42(8), 1248-1258.

- [Ghiaus, C., & Alzetto, F. (2019)](https://doi.org/10.1080/19401493.2018.1561753). Design of experiments for Quick U-building method for building energy performance measurement. Journal of building performance simulation, 12(4), 465-479.